In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import codecs
from nltk import ngrams 

stop_words = np.array(pd.read_csv('stopwords_russian2translit.txt', encoding='utf-8', lineterminator='\n')).ravel()

In [40]:
def parse_url(url, stop_words, min_len_word=1):
#   url?text#text -> url:
    temp = re.search(r'.+\?', url)
    if temp != None:
        url = temp.group(0)[:-1]
    temp = re.search(r'.+#', url)
    if temp != None:
        url = temp.group(0)[:-1]
    
#   site_name, full_domen, last_domen
    full_domen = re.search(r'\..+', url).group(0)[1:]
    if re.search(r'.+?/', full_domen) != None:
        full_domen = re.search(r'.+?/', full_domen).group(0)[:-1]    
    last_domen = full_domen.strip().split(sep='.')[-1]
    site_name = re.search(r'.+?\.', url).group(0)[:-1]
    
#   usefull_names = url.split() - domens - stop_words - set(word: len(word) < n)
    usefull_names = re.search(r'.+', url).group(0)
    if re.search(r'.+?/', usefull_names) != None:
        usefull_names = re.search(r'.+?/', usefull_names).group(0)[:-1]          
    usefull_names = set(re.split('-|_|\.|/', usefull_names))
    usefull_names = usefull_names.difference(stop_words)
    usefull_names = [item for item in usefull_names if len(item) >= min_len_word]
    usefull_names = ' '.join(list(usefull_names))

        
    return full_domen, last_domen, site_name, usefull_names

In [41]:
output_txt = 'new_sitenames.txt'
path = 'content/'

with open(output_txt, 'w') as out_f:
    out_f.write('{}\t{}\n'.format('doc_id', 'usefull_names'))
    for doc_id in range(1, 28027):
        with codecs.open(path + str(doc_id) + '.dat', 'r', 'utf-8') as in_f:
            url = in_f.readline().strip()
            _, _, _, usefull_names = parse_url(url, stop_words, min_len_word=0)
            out_f.write('{}\t{}\n'.format(doc_id, usefull_names))

In [42]:
ns = pd.read_csv('new_sitenames.txt', sep='\t', encoding='utf-8', lineterminator='\n')
ns.head(10)

,doc_id,usefull_names
0,1,zrenielib ru
1,2,online ru perevesti
2,3,timecops biz
3,4,proffi95 ru
4,5,xn p1ai jtbaaldsgaoflxr4fyc
5,6,rabotka ru
6,7,beon vladkakashi ru
7,8,ru doshkolnik
8,9,com chastnosti
9,10,pomogudengami ru


In [43]:
output_txt = 'ngrams_sitenames.txt'
un = np.array(pd.read_csv('new_sitenames.txt', sep='\t', encoding='utf-8', lineterminator='\n').usefull_names).ravel()
# un = filter(lambda v: v==v, un)
n = 3

with open(output_txt, 'w') as out_f:
    out_f.write('{}\t{}\n'.format('doc_id', 'ngrams'))
    for doc_id, it in enumerate(un):
        if not isinstance(it, np.str):
            out_f.write(str(doc_id)+'\t'+''+'\n')    
            continue
        out_f.write(str(doc_id)+'\t')    
        ngr = ngrams(it, n)
        for grams in ngr:
            out_f.write(''.join(grams) + ',')
        out_f.write('\n')